# Notebook A: Toy fragment with SKQD (MVP)

In de echte SKQD-pipeline vervang je deze directe verwachting door een Krylov-basis en GEVD op schattingen van $H$ en $S$.

In [16]:
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.transformers import FreezeCoreTransformer
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD
from qiskit.primitives import StatevectorEstimator
from qiskit.quantum_info import SparsePauliOp
from scipy.optimize import minimize
from qiskit_nature.units import DistanceUnit
import numpy as np

# H2 toy-fragment (0.74 Å) → elektronisch probleem
driver = PySCFDriver(atom='H 0 0 0; H 0 0 0.74',
                     basis='sto-3g',
                     unit=DistanceUnit.ANGSTROM)
problem = FreezeCoreTransformer().transform(driver.run())

num_particles = problem.num_particles
num_spatial_orbitals = problem.num_spatial_orbitals
mapper = ParityMapper(num_particles=num_particles)

# Fermionische Hamiltoniaan → qubitoperator (2 qubits na two-qubit reduction)
H_qubit: SparsePauliOp = mapper.map(problem.hamiltonian.second_q_op())
constant_shift = sum(problem.hamiltonian.constants.values())

# VQE-ansatz = HF + UCCSD-excitaties
hf_state = HartreeFock(num_spatial_orbitals, num_particles, mapper)
ansatz = UCCSD(num_spatial_orbitals=num_spatial_orbitals,
               num_particles=num_particles,
               qubit_mapper=mapper,
               initial_state=hf_state)

estimator = StatevectorEstimator()
parameters = list(ansatz.parameters)

def energy(theta):
    bound = ansatz.assign_parameters(dict(zip(parameters, theta)))
    ev = estimator.run([(bound, H_qubit)]).result()[0].data.evs
    return float(ev) + constant_shift  # voeg kernrepulsie toe

result = minimize(energy,
                  x0=np.zeros(len(parameters)),
                  method='COBYLA',
                  options={'maxiter': 200, 'tol': 1e-6})

print(f"VQE-energie: {result.fun:.12f} Ha")
print("Optimale parameters:", result.x)
print("Convergentie:", result.success)


VQE-energie: -1.137283834485 Ha
Optimale parameters: [ 6.71457185e-07 -1.31192358e-06 -1.12781775e-01]
Convergentie: True
